In [2]:
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.6f}".format

train_df = pd.read_csv(filepath_or_buffer="Z_expectation/Z_values_100000.csv")
test_df = pd.read_csv(filepath_or_buffer="Z_expectation/Z_values_100.csv")
validation_df = pd.read_csv(filepath_or_buffer="Z_expectation/Z_values_10000.csv")

train_df.head()

,0_0,0_1,1_0,1_1,2_0,2_1,3_0,3_1,4_0,4_1,...,7_1,8_0,8_1,9_0,9_1,decay_0,decay_1,W_0,W_1,J_0
0,1.000000,1.000000,0.820000,0.320000,0.040000,-0.340000,-0.160000,-0.200000,-0.020000,-0.080000,...,-0.720000,-0.400000,0.260000,-0.760000,0.800000,1.288213,-0.028388,1.334944,3.454920,4.611197
1,1.000000,1.000000,0.700000,0.600000,-0.040000,0.140000,-0.420000,-0.280000,-0.260000,-0.260000,...,-0.120000,-0.580000,-0.480000,-0.560000,-0.720000,0.979878,2.065299,2.823135,2.489759,3.153725
2,1.000000,1.000000,0.520000,0.400000,-0.020000,-0.400000,-0.380000,-0.380000,-0.140000,-0.100000,...,-0.320000,-0.760000,0.460000,0.200000,1.000000,0.438522,-0.289307,3.004403,3.934976,4.238538
3,1.000000,1.000000,0.640000,0.760000,-0.020000,0.380000,-0.580000,-0.460000,-0.700000,-0.720000,...,0.000000,-0.400000,-0.260000,-0.280000,-0.700000,1.591963,1.266298,3.310940,2.432493,2.903198
4,1.000000,1.000000,0.580000,0.620000,-0.360000,-0.280000,-0.440000,-0.460000,-0.300000,-0.420000,...,-0.700000,-0.200000,-0.320000,0.540000,0.080000,1.819276,0.291896,3.732673,3.377953,3.459519


In [17]:
from keras.src.layers import SimpleRNN, Dense
from keras import Sequential
from tensorflow.python.keras.regularizers import L2, L1
import numpy as np
def create_sequences(data, num_qubits, time_steps, overlap, sequence_length):
    sequences = []
    targets = []

    # Extract sequences for each qubit
    for qubit in range(num_qubits):
        qubit_sequences = []
        qubit_cols = [f'{t}_{qubit}' for t in range(time_steps)]  # Columns for this qubit

        for start in range(0, time_steps - sequence_length, sequence_length-overlap):
            end = start + sequence_length
            qubit_sequences.append(data[qubit_cols][start:end].values)
            print(start)

        sequences.append(qubit_sequences)
    targets = data[['W_0', 'W_1', 'J_0']].values  # Adjust target columns

    return np.array(sequences), np.array(targets)

In [30]:

import re
learning_rate = 0.0001
num_qubits = 2
time_steps = 10  # Adjust this based on your actual number of time steps
overlap = 4
sequence_length = 5

# Extracting Z1 and Z2 values and reshaping them into sequences
# column_names = [f'{i}_0' for i in range(time_steps)] + [f'{i}_1' for i in range(time_steps)]
# Z_values = train_df[column_names].values
# Z_values = Z_values.reshape(-1, time_steps, num_features)
#targets = train_df[['W_0', 'W_1', 'J_0']].values

z_sequences, target_sequences = create_sequences(train_df, num_qubits, time_steps, overlap, sequence_length)

0
1
2
3
4
0
1
2
3
4


In [34]:


# Extracting target values

# Now, Z_values is your input for the RNN and targets are the values you're trying to predict

# Define the RNN model
model = Sequential()
model.add(SimpleRNN(50, activation='relu', input_shape=(5, 5, 10)))  # Adjusted input shape
model.add(SimpleRNN(50, activation='relu'))
model.add(Dense(3))  # Output layer with 3 units for W_0, W_1, and J_0

# Compile the model
# optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
# model.compile(optimizer=optimizer, loss='mean_squared_error')
model.compile(optimizer='adam', loss='mean_squared_error')



ValueError: Input 0 of layer "simple_rnn_9" is incompatible with the layer: expected ndim=3, found ndim=4. Full shape received: (None, 5, 5, 10)

In [33]:
# Training parameters
epochs = 100
batch_size = 1000

# Train the model
history = model.fit(z_sequences, target_sequences, epochs=epochs, batch_size=batch_size, validation_split=0.2)
# Extract loss and validation loss
train_loss = history.history['loss']
val_loss = history.history['val_loss']

Epoch 1/100


ValueError: in user code:

    File "C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\

In [ ]:
epochs_range = range(epochs)

# Plotting
plt.figure(figsize=(8, 8))
plt.plot(epochs_range, train_loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Over Epochs')
plt.legend()
#plt.xlim([1,epochs])
#plt.ylim([0,2])
plt.semilogy()
plt.show()